In [9]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

from engine.globalconfig import GlobalConfig
from engine.loader.portfolioload import PortfolioLoader
from engine.loader.curveload import MarketDataLoader
from engine.loader.indexload import MarketIndex
from datetime import date, datetime
import tquant as tq

# Creazione del portafoglio

In [3]:
#Tutti gli errori sono definiti per benchmark. Sono errori di mancanza di indici nella libreria o di dati nel file json
config = GlobalConfig.from_json()
# TODO caricare indici in memoria con fixing storici 
ptl = PortfolioLoader(config)

# Market Data

In [4]:
config = GlobalConfig.from_json()
mdl = MarketDataLoader(config)

evaluation_date = tq.Settings.evaluation_date

# TODO da fare bootstrapping
rates = mdl.ir_eur_curve_estr['quote'].values
times = mdl.ir_eur_curve_estr['daycount'].values/365

rates_6m = mdl.ir_eur_curve_6m['quote'].values
times_6m = mdl.ir_eur_curve_6m['daycount'].values/365

estr_curve = tq.RateCurve(times, rates)
eur6m_curve = tq.RateCurve(times_6m, rates_6m)

# Caricamento dei past fixings
MarketIndex()._load_data(config)

curve = MarketIndex()._load_curves(mdl)


In [6]:
trade = ptl.portfolio["0-98295668"]

# TODO se vogliamo fare con un pricer generico vanno cambiate alcune cose, tra cui mettere la curva dell'euribor nella creazione dell'indice

if trade.product_type == 'InterestRateSwap':
    engine = tq.SwapAnalyticEngine(trade)
    npv_swap = engine.price(estr_curve, curve[trade.floating_leg.index.name], evaluation_date)

npv_swap

<tf.Tensor: shape=(), dtype=float64, numpy=-6087.221064957557>

In [5]:
floating_leg_pricer = tq.FloatingLegDiscounting(trade.floating_leg)
fixed_leg_pricer = tq.FixedLegDiscounting(trade.fixed_leg)

npv_float = floating_leg_pricer.price(estr_curve, eur6m_curve, evaluation_date, tq.FloatingCouponDiscounting)
npv_fixed = fixed_leg_pricer.price(estr_curve, evaluation_date, tq.FixedCouponDiscounting)

In [6]:
print(f"NPV FixedLeg: {npv_fixed :,.0f}")
print(f"NPV FloatingLeg: {npv_float :,.0f}")
########
print(f"NPV Swap: {npv_swap :,.0f}")

NPV FixedLeg: 579,779
NPV FloatingLeg: -582,160
NPV Swap: -6,087


In [10]:
#Pricing portfolio
results = {}
notional = {}
check = {}

for tradenum in ptl.portfolio:
    trade = ptl.portfolio[tradenum]
    try:
        if trade.product_type == 'InterestRateSwap':
            if trade._currency == tq.Currency.EUR.name:
                engine = tq.SwapAnalyticEngine(trade)
                npv_swap = engine.price(estr_curve, curve[trade.floating_leg.index.name], evaluation_date)
                results[tradenum] = npv_swap.numpy()
                
                notional[tradenum] = np.max(np.abs(trade.fixed_leg.notionals))

                # controllo che i vettori di nozionali siano uguali tra loro
                check[tradenum] = (np.array_equiv(trade.fixed_leg.notionals,notional[tradenum]) & np.array_equiv(np.abs(trade.floating_leg.notionals),notional[tradenum]))

            else:
                results[tradenum] = 'Different Currency'
                notional[tradenum] = 'Different Currency'
                print(trade._currency)
    
    except Exception as error:
        results[tradenum] = error
        notional[tradenum] = error

In [11]:
isp = pd.read_csv('../data/20240430/output/npv_intesa.csv')
#isp.dropna(inplace=True)

def dateCheck(datestring, eDate):
    return datetime.strptime(datestring, '%Y/%m/%d').date() > eDate

trade_id = []
npvs = []
npvs_isp = []
stard_date = []
notionals = []
checks = []


for key in results.keys():
    
    if isinstance(results[key], np.floating):
        trade_id.append(key)
        npvs.append(results[key])
        notionals.append(notional[key])
        checks.append(check[key])

        try: 
            npvs_isp.append((isp[isp["DEALID"]==key]["NPV"]).to_numpy()[0])
            #check =  dateCheck(datestring= (isp[isp["DEALID"]==key]["START_DATE"]).to_list()[0], eDate=evaluation_date)
            #stard_date.append(check)
        except:
            npvs_isp.append(np.NAN)

r = pd.DataFrame(data= [trade_id, npvs, npvs_isp, notionals,checks], index=["Trade_id", "NPV iRE", "NPV ISP", "Notional (Max Fixed Leg)", "Constant Notional"]).T


r.dropna(inplace =True)
r["Abs Delta $"] = abs(r["NPV iRE"]-r["NPV ISP"])
r["Abs Delta $ / ISP"] = abs(r["NPV iRE"]-r["NPV ISP"])/abs(r["NPV ISP"])
r["Abs Delta $ / Notional"] = abs(r["Abs Delta $"])/abs(r["Notional (Max Fixed Leg)"])
r.sort_values(by=["Abs Delta $ / ISP"], ascending=False, inplace=True)
#r.sort_values(by=["Abs Delta $"], ascending=False, inplace=True)
r.reset_index(inplace =True, drop = True)


def highlighter(x):
    is_higher_than = x > 0.1
    return ["color: #EE2E31" if i else "color: #CCCCCC" for i in is_higher_than]

In [21]:
import json

def additional_flows(trade, evaluation_date):
   date_format = '%Y-%m-%d'
   leg1_schedule_len = len(trade['data']['Leg1.additionalFlowsTable']['rows'])
   #print(trade['data']['Leg1.isCap'], trade['data']['Leg1.isFloor'])  #--> no cap/floor on both legs
   #print(trade['data']['Leg2.isCap'], trade['data']['Leg2.isFloor'])
   
   leg1_additionalFlows = []
   leg2_additionalFlows = []

   if leg1_schedule_len != 0:
      
      for i in range(leg1_schedule_len):
         date_check = datetime.strptime(trade['data']['Leg1.additionalFlowsSchedule']['rows'][i]['paymentDate'], date_format).date()
         
         if date_check > evaluation_date:
            leg1_additionalFlows.append(trade['data']['Leg1.additionalFlowsTable']['rows'][i]['value'])

      sum1 = sum(leg1_additionalFlows)


      try:
         leg2_schedule_len = len(trade['data']['Leg2.additionalFlowsSchedule']['rows'])
         leg2_additionalFlows = []
      
         for i in range(leg2_schedule_len):
            date_check = datetime.strptime(trade['data']['Leg2.additionalFlowsSchedule']['rows'][i]['paymentDate'], date_format).date()
            if date_check > evaluation_date:
               leg2_additionalFlows.append(trade['data']['Leg2.additionalFlowsTable']['rows'][i]['value'])

         sum2 = sum(leg2_additionalFlows)
      except:
         sum2 = 0.0
   else:
      sum1 = 0
      sum2 = 0
   
   return sum1, sum2

file_name = "../data/20240430"
local_data = [file_name + "/trades/TradesAggregated_" + str(i) + ".json" for i in range(1, 4)]

x = []

for filename in local_data:
   with open(filename, "r") as file:   
      x.append(json.load(file))

js1 = x[0]
js2 = x[1]
js3 = x[2]
combined_data = js1 + js2 + js3


trade_id = []
add_flows_leg1 = []
add_flows_leg2 = []

for tradenum in combined_data :
   if tradenum['data']['tradeRef'] in r['Trade_id'].values:
      af1, af2 = additional_flows(tradenum, evaluation_date)
      trade_id.append(tradenum['data']['tradeRef'])
      add_flows_leg1.append(af1)
      add_flows_leg2.append(af2)


additional_flowss = pd.DataFrame({'Trade_id': trade_id, 'Sum of additional flows leg 1': add_flows_leg1, 'Sum of additional flows leg 2': add_flows_leg2})


IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFixed IRFixed
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFixed IRFixed
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat IRFloat
IRFloat 

In [13]:
r = r.merge(additional_flowss)

styled_r = r.style.format({
      "NPV iRE": "{:,.3f}",
      "NPV ISP": "{:,.3f}",
      "Abs Delta $": "{:,.3f}",
      "Abs Delta $ / ISP": "{:.2%}",
      "Notional (Max Fixed Leg)": "{:,.3f}",
      "Sum of additional flows leg 1": "{:,.3f}",
      "Sum of additional flows leg 2": "{:,.3f}",
      "Abs Delta $ / Notional": "{:.2%}"
   }).apply(highlighter, subset="Abs Delta $ / ISP")

styled_r

,Trade_id,NPV iRE,NPV ISP,Notional (Max Fixed Leg),Constant Notional,Abs Delta $,Abs Delta $ / ISP,Abs Delta $ / Notional,Sum of additional flows leg 1,Sum of additional flows leg 2
0,0-98295668,"-6,087.221","-2,103.665","10,000,000.000",False,"3,983.556",189.36%,0.04%,0.000,0.000
1,0-92835077,"-524,914.991","644,581.840","10,000,000.000",False,"1,169,496.830",181.43%,11.69%,0.000,0.000
2,0-100690218,"13,341.419","-27,503.934","12,844,615.380",True,"40,845.353",148.51%,0.32%,0.000,0.000
3,21773899-35900449,"1,366.721",552.116,"2,751,708.400",False,814.605,147.54%,0.03%,0.000,0.000
4,27019039-41824345,"-124,073.048","-51,986.148","3,916,140.690",False,"72,086.901",138.67%,1.84%,"70,313.040",0.000
5,21775321-35901539,"34,127,910.640","17,525,741.795","4,728,408.236",False,"16,602,168.845",94.73%,351.12%,0.000,0.000
6,25547007-40374896,"-556,544.646","-325,672.739","29,500,000.000",False,"230,871.906",70.89%,0.78%,0.000,0.000
7,0-95460840,"-198,765.222","-116,590.671","7,350,000.000",False,"82,174.550",70.48%,1.12%,0.000,0.000
8,0-92835679,"-1,053.965",-650.801,"1,961,043.410",False,403.164,61.95%,0.02%,0.000,0.000
9,0-92835566,"-7,367.273","-4,549.133","13,707,805.770",False,"2,818.140",61.95%,0.02%,0.000,0.000


In [14]:
#styled_r.to_excel("Results.xlsx")